# Prédiction de la qualité d'un film à partir de ses caractéristiques

## Présentation du projet
Le but de notre projet est de créer un modèle économétrique pour prédire le succès d'un film à partir de ses caractéristiques, comme sa durée, son genre, son réalisateur, etc. 

Nous avons décidé d'utiliser les données d'IMDb, un site de notation et de référencement des films. L'avantage de cette plateforme est qu'elle permet aux internautes de noter les films qu'ils ont vu, ce qui sera ce que l'on considère comme la mesure du succès d'un film. Par ailleurs, de nombreuses informations qui nous seront utiles sont présentes sur ce site et sont plus directement accessibles que sur une plateforme comme Wikipedia.

## Commençons par la mise en place du jeu de données
Nous importons une base mise à disposition par IMDb, que nous rendons exploitable par quelques opérations élémentaires.
Nous ne conservons que les films qui ont 2.000 votes ou plus : cela permet d'une part d'éviter de considérer les films trop peu votés pour que leur note moyenne soit pertinente, et d'autre part d'avoir un jeu de données plus léger.

In [ ]:
import pandas as pd
movies_rating = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz")
movies_rating_clean = movies_rating["tconst\taverageRating\tnumVotes"].str.split("\\t", expand=True)
movies_rating_clean.columns = ['ID', 'Note_moyenne', 'Nombre_de_votes']
movies_rating_clean['Note_moyenne'] = movies_rating_clean['Note_moyenne'].astype(float)
movies_rating_clean['Nombre_de_votes'] = movies_rating_clean['Nombre_de_votes'].astype(float)
movies_rating_filtré = movies_rating_clean[movies_rating_clean.Nombre_de_votes > 1999]

**Nous voilà maintenant en possession d'une première base de travail**

Le problème de ce jeu de donné téléchargé, c'est qu'il ne contient que des informations sur les votes des films. Il n'y a aucune mention d'autres caractéristiques dont nous pourrons avoir besoin, comme son cast. Nous avons contacté les services d'IMDb, mais leur API est payant... Nous avons donc choisi de scraper les informations dont nous avons besoin.

Cela dit, la base téléchargée va nous être particulièrement utile ! Nous avons à disposition les identifiants de tous les films de la plateforme qui ont recueilli 2000 votes ou plus, et l'URL des pages des films s'écrit à partir de cet identifiant.

Après avoir essayé de scraper les éléments en cherchant des chemins d'accès manuellement dans le code HTML, nous avons trouvé dans chaque page un dictionnaire qui comprend les caractéristiques principales des films. Le code suivant permet de recueillir ces données, les traiter pour les rendre exploitables, et les insérer dans un dataframe.

In [ ]:
#On fabrique le squelette du dataframe que l'on va remplir au fur et à mesure du scrap ; on le fusionnera par la suite
#avec la database téléchargée plus tôt
#On indique le nom des colonnes, qui sont les variables que l'on choisit de conserver
#contentRating est la classification d'age, creator est la société de production
df = pd.DataFrame(columns=['name', 'alternateName', 'url', 'contentRating', 'datePublished', 'genre', 'actor', 'director', 'creator', 'Origine', 'Budget', 'duration', 'keywords'])

In [ ]:
from bs4 import BeautifulSoup as bs
import requests
from random import seed
import time
import json
import re
from google.colab import files


#C'est une liste des objets inutiles dans le scrap des pages ; je retire aussi le contenu du dataframe qu'on a déjà (les votes)
superflu = ["@context", "@type", "image", "description", "review", "trailer", "aggregateRating"]
#Celui-ci servira à retirer les images des scénaristes, etc
superflu2 = ['@type', 'url']

session_obj = requests.Session()


#On boucle sur chaque film qu'on considère
#Le compteur est cosmétique : il sert à nous rassurer sur le fait que tout se passe bien pendant le scrap
compteur = 0
for ID in movies_rating_pur['ID'] :
  compteur = compteur+1
  print(compteur)
  try: #On utilise un try except au cas où on aurait un problème sur une page : on ne veut pas que l'exécution s'arrête après des heures de scrap
    time.sleep(0.01) #On ajoute un petit délai pour ne pas surcharger le site de requêtes
    url_temp = 'https://www.imdb.com/title/'+ID+'/'
    response=session_obj.get(url_temp, headers={"User-Agent": "Mozilla/5.0"}) #On se fait passer pour une session normale ;) 
    html = response.content
    soup = bs(html, "html.parser")
    
    #Le bloc est composé de la partie de chaque page qui contient les informations utiles
    #On le transforme en dictionnaire
    bloc = soup.find("script", type="application/ld+json").string
    dictio = json.loads(bloc)
    
    #On retire dedans ce qui ne nous intéresse pas
    for inutile in superflu :
      dictio.pop(inutile, None)

    #on ajoute une ligne budget illico presto ATTENTION CA NE MARCHE PAS
    liste_budg = soup.find_all("label", class_="ipc-metadata-list-item__list-content-item")
    if len(liste_budg) >= 3 and '$' in liste_budg[2] :
      budget = liste_budg[2].string
      if budget == None :
        budget = "Non renseigné"
      else :
        budget = "".join([elemnt for elemnt in budget if elemnt.isdigit()])
      dictio['Budget'] = budget

    #L'allure du dictionnaire n'est pas parfaitement satisfaisante, par exemple chaque acteur est associé à une date de naissance,
    #à une photo, etc... On ne conserve que le nom des acteurs, et celui des réalisateurs
    
    if 'actor' in dictio :
      for acteur in dictio['actor'] :
        for inutile in superflu2 :
          acteur.pop(inutile, None)
      for indice, nom in enumerate(dictio['actor']) :
        dictio['actor'][indice] = nom['name']

    if 'director' in dictio :
      for directeur in dictio['director'] :
        for inutile in superflu2 :
          directeur.pop(inutile, None)
      for indice, nom in enumerate(dictio['director']) :
        dictio['director'][indice] = nom['name']
    
    if 'creator' in dictio :
      for createur in dictio['creator'] :
        createur.pop('@type', None)
      for indice, url in enumerate(dictio['creator']) :
        dictio['creator'][indice] = url['url']

    #Pour la société de production c'est un peu compliqué : on n'a qu'une URL
    #Ce qui n'est pas grave, puisqu'on peut retrouver son nom en scrapant cet url !
    #Mais le temps d'exécution explose si on le fait ; j'inclus donc ce code (complètement fonctionnel)
    #mais en pratique il prend trop de temps à tourner
    
    if 'creator' in dictio :
      for index, createur in enumerate(dictio['creator']) :
        url_temp = 'https://www.imdb.com'+createur
        response=session_obj.get(url_temp, headers={"User-Agent": "Mozilla/5.0"})
        html = response.content
        soup = bs(html, "lxml")
        compagnie_soup = soup.find("title")
        if compagnie_soup == None :
          compagnie = "Non renseigné"
        else :
          compagnie = compagnie_soup.string
        compagnie = compagnie[5:-40] #on garde que l'élément important du titre
        dictio['creator'][index] = compagnie

    #On ajoute au dictionnaire le pays d'origine, que l'on le trouve dans la date de sortie
    date_sortie_soup = soup.find("a", class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link", href="/title/"+ID+"/releaseinfo?ref_=tt_dt_rdat")
    if date_sortie_soup == None :
      date_sortieV2 = "Non renseigné"
      pays = "Non renseigné"
    else :
      date_sortieV2 = date_sortie_soup.string
      b1 = date_sortieV2.find('(')
      b2 = date_sortieV2.find(')')
      pays = date_sortieV2[b1:b2]
      pays=pays[1:]
    dictio['Origine'] = pays

    #On ajoute dans le dataframe la ligne qui correspond au film
    df = df.append(dictio, ignore_index=True)
    sauvegarde_imdb = df.to_csv('IMDB_2000votes.csv', index = True) #Et par précaution, on sauvegarde le dataframe à chaque itération
  except:
    print('Erreur au rang : '+str(compteur))

**Et voilà : nous avons construit une base de donnée grâce au scraping**

Je ne conseille pas de lancer ce code, puisqu'il nous a fallu plus d'une journée pour obtenir le dataframe, sans compter toutes les fois où notre ordinateur a eu des problèmes de connexion et interrompu l'exécution (ce qui nous a coûté au total 3 jours). En revanche, il est possible de le lancer sur quelques valeurs, pour obtenir un échantillon.

Nous mettrons à disposition le dataframe complet, pour pouvoir lancer le reste du code sans scraper à nouveau ces quelques 46.000 pages.